In [2]:
import numpy as np
import pandas as pd
from scipy.spatial.distance import cosine

import torch
from transformers import BertModel, BertTokenizer

import matplotlib.pyplot as plt
import seaborn as sn

## Example sentences

In [ ]:
sentences = ["After stealing money from the bank vault, the bank robber was seen fishing on the Mississippi river bank."]



## Encode words

In [ ]:
def encode_sentences(model_name_or_path, sentences):
    """
    Given a list of sentences an d a model, get the embeddings of theses sentences
    as the average of the word embeddings of the last layer.
    """
    print("   Loading pretrained model/tokenizer...")
    tokenizer = BertTokenizer.from_pretrained(model_name_or_path)
    model = BertModel.from_pretrained(model_name_or_path, output_hidden_states=True, cache_dir ='../_cache') # Will output all hidden_states.

    print("   Tokenizing sentences...")
    tokenized = [tokenizer.encode(sent, add_special_tokens=True) for sent in sentences]

    max_len = 0
    for i in tokenized:
        if len(i) > max_len:
            max_len = len(i)
    print("   Maximum length in dataset: {}".format(max_len))

    print("   Padding/Truncating sentences according to the maximum length...")
    padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized])

    print("   Creating attention masks...")
    attention_mask = np.where(padded != 0, 1, 0)  #returns ndarray which is 1 if padded != 0 is True and 0 if False.

    print("   Converting inputs to torch tensors...")
    input_ids = torch.tensor(padded)  
    attention_mask = torch.tensor(attention_mask)

    print("   Encoding sentences...")
    with torch.no_grad():
        # output is a 2-tuple where:
        #  - output[0] is the last_hidden_state, i.e a tensor of shape (batch_size, sequence_length, hidden_size).
        #  - output[1] is the pooler_output, i.e. a tensor of shape (batch_size, hidden_size) being the last layer hidden-state of the first token of the sequence (classification token).
        #  - output[2] are all hidden_states, i.e. a 13-tuple of torch tensors of shape (batch_size, sequence_length, hidden_size): 12 encoders-outputs + initial embedding outputs.
        output = model(input_ids, attention_mask=attention_mask)

    # Concatenate the tensors for all layers. We use `stack` here to create a new dimension in the tensor.
    hidden_states = torch.stack(output[2], dim=0)

    # Switch around the “layers” and “tokens” dimensions with permute.
    hidden_states = hidden_states.permute(1,2,0,3)

    # For each sentence, take the embeddings of its word from the last layer and represent that sentence by their average.
    last_hidden_states = output[0]
    sentence_embeddings = [torch.mean(embeddings, dim=0).numpy() for embeddings in last_hidden_states]
    sentence_embeddings = np.array(sentence_embeddings)

    # Create pandas dataframe.
    cols = ['feat'+str(i) for i in range(sentence_embeddings.shape[1])]
    df = pd.DataFrame(data=sentence_embeddings[:,:], columns=cols)
    df['Sentence'] = sentences
    return df

In [ ]:
# Tokenization in multiple steps.
marked_text = "[CLS] " + sentence + " [SEP]"
tokenized_text = tokenizer.tokenize(marked_text)
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

# Tokenization in one step.
tokenized = tokenizer.encode(sentence, add_special_tokens=True)
tokenized = torch.tensor([tokenized])

# Encode it.
with torch.no_grad():
    output = model(tokenized)
    
# Get all hidden states.
hidden_states = output[2]
hidden_states = torch.stack(hidden_states, dim=0)
hidden_states = torch.squeeze(hidden_states, dim=1)
hidden_states = hidden_states.permute(1,0,2)
print(" - 'hidden_states' is a {} tensor.".format(hidden_states.size()))

# Sum the last four layers.
token_vecs_sum = []
for token in hidden_states:
    sum_vec = torch.sum(token[-4:], dim=0)
    token_vecs_sum.append(sum_vec)
print(" - 'token_vecs_sum' is a ({}, {}) array.".format(len(token_vecs_sum), len(token_vecs_sum[0])))

# Get idx of words of interest 'bank'.
for i, token_str in enumerate(tokenized_text):
    print(i, token_str)

# Calculate the cosine similarity between the word bank in "bank robber" vs "river bank" (different meanings).
diff_bank = 1 - cosine(token_vecs_sum[10], token_vecs_sum[19])

# Calculate the cosine similarity between the word bank in "bank robber" vs "bank vault" (same meaning).
same_bank = 1 - cosine(token_vecs_sum[10], token_vecs_sum[6])

print('Vector similarity for  *similar*  meanings:  %.2f' % same_bank)
print('Vector similarity for *different* meanings:  %.2f' % diff_bank)

In [ ]:
# List Mark
DNS
mDNS

LAN
WAN
CAN
MAN

ACL
PACL
VACL

HSRP
VRRP
GLBP

EIGRP
IGRP
BGP
EBGP

STP
DTP
VTP